# 4. GPT-2

In [ ]:
import torch
import torch.nn as nn
from torchtext.data.utils import get_tokenizer

In [ ]:
tokenizer = get_tokenizer('basic_english')

tokenizer(corpus_string)[:20]


from torchtext.vocab import build_vocab_from_iterator
vocab = build_vocab_from_iterator([tokenizer(corpus_string)])

In [ ]:
from torchtext.vocab import build_vocab_from_iterator
vocab = build_vocab_from_iterator([tokenizer(corpus_string)], specials=['<unk>'])

In [ ]:
from pathlib import Path

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers==4.2.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 22.3 MB/s 
     |████████████████████████████████| 2.9 MB 37.8 MB/s 
     |████████████████████████████████| 880 kB 40.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=a731af7db8e3fa70b83418b9ad75e34fd8d8f8fe775f4b2e30f2261848ffc373
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.19.2
    Uninstalling transformers-4.19.2:
      Successfully uninstalled transformers-4.19.2


In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

train_path = '/content/gdrive/MyDrive/3gr_1/New/train-data.txt'
test_path = '/content/gdrive/MyDrive/3gr_1/New/test-data.txt'

In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("gpt2")


training_args = TrainingArguments(
    output_dir="./gpt2-gerchef", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=9, training_loss=6.150740729437934, metrics={'train_runtime': 518.1963, 'train_samples_per_second': 0.017, 'total_flos': 23796873363456, 'epoch': 3.0})

In [ ]:
trainer.save_model()

In [ ]:
# train된 코드로 text generate 하는 것

In [ ]:
from transformers import pipeline

generate = pipeline('text-generation',model='./gpt2-gerchef', tokenizer='gpt2',config={'max_length':20000})

In [ ]:
generate('My name is Emma and I am')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'My name is Emma and I am a teacher at the Catholic university in Wappingham. She has just finished school and we are having a great time.‼\nIt was quite an evening, and as you can see I am a regular student'}]

In [ ]:
generate('My name is John and I am')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "My name is John and I am an atheist. I believe there may be some other things I can do to help myself with things than try and do them all. And as much as I'd have to put up with many of the things I've"}]

In [ ]:
generate('My name is James and I am')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'My name is James and I am coming to you tonight to give you an update on who is the last person for whose head he fell with a gunshot which led to more than 11 dead in Oakland, California earlier this month. It looks like you need'}]

In [ ]:
generate('My name is Olivia and I am')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'My name is Olivia and I am also in this world! I have been looking for my mother and I have just arrived in New Zealand so I must tell you all about my new found love so I can finally get to experience all the wonderful things that'}]

In [ ]:
generate('My name is Sophia and I am')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'My name is Sophia and I am from the Philippines. You must always keep your best interests at heart when planning an assignment." – Sophia (sophia)\n\n"One day we are told to bring one a piece of cloth with us.'}]

In [ ]:
generate('My name is Ji young and I am')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'My name is Ji young and I am from Dongguan University," said the university\'s president Zhang Yang. "This is a high school student\'s home. It is the first time that I have been accepted to a university\'s university." Huang was'}]

In [ ]:
generate('Girl is')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Girl is more intelligent, more compassionate and just when she has that, you know something's very wrong (laughs)! But like for my daughter that's the hardest thing.\n\nHARRY O'DONNELL: My daughter and I have"}]

In [ ]:
generate('Boy is')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Boy is really just like his dad. He\'s a guy who has the best chance of landing a job in front of a bunch people, so that could get him a big promotion and start making money."\n\nAccording to Bloomberg, the New York'}]

In [ ]:
generate('Boy is')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Boy is an example of a good writer. She had an extremely powerful narrative and developed something that resonated with his listeners that we all needed to hear. She has an amazing ability to connect with her listeners, create great conversation, to connect with them'}]

In [ ]:
generate('She is')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'She is a former journalist who wrote for the Atlantic and Newsweek. Her novels about science have appeared in several anthologies including Wired, Salon and Slate. She is listed on the cover of the American Conservative Monthly Magazine. She lives in Manhattan.'}]

In [ ]:
generate('He is')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'He is also trying to prove that we can safely go on our own, because there is literally no way for us to continue to live with these injuries."\n\nThe only other person still injured in the incident was a boy who was found slumped over'}]

In [ ]:
generate('My lover is')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'My lover is a goddess and her beauty takes on a life of its own. His son does not leave his wife and is the only daughter.\n\n"I want to do something. I want to leave my brother to grow up as one of'}]

In [ ]:
generate('My boss is')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'My boss is running the ship. He will take over by any means necessary, and he must act decisively to prevent the attack.\n\n\nWhat if everyone is in the same class or class and he has 3 skill points to spend?\n\nThe'}]